# Start client

In [2]:
import dask.dataframe as dd
from dask.distributed import Client

#Initializing client
client = Client()
client

/anaconda3/envs/overscripted/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/anaconda3/envs/overscripted/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


Client Scheduler: tcp://127.0.0.1:51384 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.59 GB


# Data
Using filtered and evaluated for json data named 'is_json_above_mean.parquet'. You can get this by running the 'isJson_dataPrep.ipynb'

This new sample has 499805, meaning that its only 4,42% of the original sample (most values are smaller than the sample's mean of 1356)

In [3]:
df = dd.read_parquet('is_json_above_mean.parquet')
df.head()

,value_1000,value,value_len,symbol,script_url,is_json
0,"{""im-settings"":""{\""val\"":{\""settings\"":{\""Site...","{""im-settings"":""{\""val\"":{\""settings\"":{\""Site...",3713,window.sessionStorage,https://assets.adobedtm.com/caacec67651710193d...,True
1,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...","{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",103878,window.localStorage,https://g.alicdn.com/alilog/mlog/aplus_v2.js,True
2,"{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...","{""APLUS_S_CORE_0.17.12_20171214163401_2ee09a0c...",103878,window.localStorage,https://g.alicdn.com/alilog/mlog/aplus_v2.js,True
3,usunico=17/12/2017:0-00155123:830; SessionASM=...,usunico=17/12/2017:0-00155123:830; SessionASM=...,1358,window.document.cookie,https://www.coches.net/scripts/common.min.js?2...,False
4,usunico=17/12/2017:0-00155123:830; SessionASM=...,usunico=17/12/2017:0-00155123:830; SessionASM=...,1358,window.document.cookie,https://www.coches.net/scripts/common.min.js?2...,False


## DF overview
Some overview about the sample after the data prep: 
- Rows: 499805
- Mean: 27829.33,
- Min: 1357,
- Max: 4496861
- Std: 122092.41

In [4]:
df_mean = df['value_len'].mean()
df_min = df['value_len'].min()
df_max = df['value_len'].max()
df_std = df['value_len'].std()
(df_mean, df_min, df_max, df_std) = dd.compute(df_mean, df_min, df_max, df_std);
print(df_mean, df_min, df_max, df_std)

27829.332847810645 1357 4496861 122092.41371885882


# Quantity analysis 

## Whole
This whole sample has: 
- False: 307577 rows
    - 61,54% are not valid JSON
    
    
- True: 192228 rows
    - 38,46% are valid JSON

In [7]:
df['is_json'].value_counts().compute()


False    307577
True     192228
Name: is_json, dtype: int64

## ORIGINAL SAMPLE: One std above the mean
Original Sample Data: 
- Mean: 1356.97
- Std: 26310.62

I'll be using the original sample's mean and std to make the following analyses


In [14]:
ROW_COUNT = 499805
MEAN = 1356
STD = 26310


- 46745 rows have the value_len greater than 27666
- This represent 9.35% of the rows on this sample

As the value_len increases the percentage of valid JSON on the columns 'value' also increases, for this filtered sample the following data was verified: 
- True: 46691 rows
    - 99,88% are valid JSON
    
- False: 54 rows
    - 0,11% are not valid JSON


In [22]:
dfa = df[df['value_len'] > (MEAN + STD)]
length = len(dfa)
print("len: {0} ({1:0.2f}%)".format(length, length / ROW_COUNT * 100))

len: 46745 (9.35%)


In [23]:
dfa['is_json'].value_counts().compute()

True     46691
False       54
Name: is_json, dtype: int64

## All greater values are JSON

There is absolute no value greater than 104653 that represents a valid JSON. 

This implies that all the greater values are JSON but they represent very low percentage of the whole data. 

The top 46745 gratest value_len are valid JSONs, that is 9.35% of this sample (value_len > mean) and 0,41% of the original sample with all the smaller values. 

In [34]:
group = dfa.groupby('is_json')
group_result = group.agg({'value_len': ['mean', 'std', 'min', 'max', 'count']}).compute()

In [36]:
group_result

value_len                                      
                  mean            std    min      max  count
is_json                                                     
False     82460.055556   13627.118063  28135   104653     54
True     271422.740185  412552.298613  27669  4496861  46691

In [37]:
allJson = df[df['value_len'] > 104653]
length = len(dfa)
print("len: {0} ({1:0.2f}%)".format(length, length / ROW_COUNT * 100))

len: 46745 (9.35%)
